词向量：
1. 离散式表示：one-hot编码，每一个单词为一个词向量，向量长度为整个词汇表。如,[0,0,0,0,1,0,0]
2. 分布式表示。nn.Embedding(a,b)，a表示单词的个数，b表示词向量宽度，通过训练学习得到词向量，原理依据就是每一个词的出现的概率取决于
   其上下文中词的概率的平均。
自然语言模型：表示一个句子出现的概率，p(w1,w2,w3,,,wn)=p(w1)*p(w2|w1)*p(w3|w1,w2),,,p(wn|w1,w2,,,wn-1)，这种预测一个序列的概率输出，完全可以用RNN这种模型来做，因为RNN就是输入一个序列，通过计算（根据当前时刻输入和上一个时刻输出计算当前时刻输出），得到一个序列。

本次要学习的东西:
一个数据处理的包 torchtext
如何搭建，RNN，LSTM，GRU等模型。
包括RNN训练的技巧等、保存和读取模型等。
###：模型输入是一串文字，输出也是一串文字，他们之间相差一个位置，语言模型的目标是根据之前的单词预测下一个单词，这就可以解释了
我们通常把整个序列作为一个训练样本，所以总的误差就是每一步的误差的加和。
RNN其实是很浅层的网络（目前见过的只有两三层的），但是却是很宽（通常一个序列会很长）的网络！！！

In [1]:
import torchtext #这是处理数据的一个包
import torch
import random
from torchtext.vocab import Vectors
import numpy as np
USE_CUDA=torch.cuda.is_available()

random.seed(123)
np.random.seed(123)
if USE_CUDA:
    torch.cuda.manual_seed(123)
BATCH_SIZE=32
EMBEDDING_SIZE=650#词向量的长度
MAX_VOCAB_SIZE=50000

In [2]:
TEXT=torchtext.data.Field(lower=True)#Field决定了如何处理数据，这相当于一个工具。lower表示，所有的单词会被小写。
train,val,test=torchtext.datasets.LanguageModelingDataset.splits(path='/home/control/Desktop/text8',
                    train='text8.train.txt',validation='text8.dev.txt',test='text8.test.txt',text_field=TEXT)
#torchtext提供了LanguageModelingDataset这个类帮助处理数据集。
print(train)

In [3]:
print(type(train))
print(len(train))

<class 'torchtext.datasets.language_modeling.LanguageModelingDataset'>
1


In [4]:
TEXT.build_vocab(train,max_size=MAX_VOCAB_SIZE)#build_vocab根据提供的训练集创建最高频的单词表，max_size限定单词数量。

In [5]:
print('vocabulary size :{}'.format(len(TEXT.vocab)))

vocabulary size :50002


BPTTIterator可以连续地得到连贯的句子，BPTT的全程是back propagation throught time,就是在RNN中使用的反向传播算法，可以得到
输出对U，V，W三个的求导。

In [6]:
VOCAB_SIZE=len(TEXT.vocab)
print(VOCAB_SIZE)

50002


总的词汇表的长度是50000，按照之前的方式生成的词向量是50000维的，这样的是非常稀疏的，在这里的，使用的EMBEDDING_SIZE=650作为词向量的
长度！，所以这里学习了一个embedding层，将50000映射成650，即学习了一个二维矩阵，(50000,650).

In [7]:
DEVICE=torch.device('cuda')
print(DEVICE)

cuda


In [8]:
train_iter,val_iter,test_iter=torchtext.data.BPTTIterator.splits((train,val,test),batch_size=BATCH_SIZE,
                            device=DEVICE,bptt_len=32,repeat=False,shuffle=True)
print(train_iter)#是一个迭代器

可迭代对象（可用于for的）包含两类：一类是dict，list，tuple，str，set等。一类是generator,迭代器。迭代器不仅可用于for循环，还可以放入next中不断的取回下一个值。普通的dict、list等放入iter()中就是变成了迭代器.

In [9]:
it=iter(train_iter)
batch=next(it)
print(' '.join([ TEXT.vocab.itos[i] for i in batch.text[:,31].data]))
#print(batch.text[:,1].data)
print(' '.join([ TEXT.vocab.itos[i] for i in batch.target[:,31].data]))

a rather fast decision one year or more is not unusual people who have only changed their name have a questionable legal status while most of the time this is perfectly sufficient
rather fast decision one year or more is not unusual people who have only changed their name have a questionable legal status while most of the time this is perfectly sufficient there


In [51]:
# for i in range(32):
#     print(' '.join([ TEXT.vocab.itos[i] for i in batch.text[:,i].data]))
#     #print(batch.text[:,1].data)
#     print(' '.join([ TEXT.vocab.itos[i] for i in batch.target[:,i].data]))

In [52]:
#yield 生成器。含有yield的就不是一个函数。而是一个生成器了。
# def foo ():
#     print('*'*20)
#     while True:
#         res= yield 4
#         print("res:",res)
# g=foo()
# print(next(g))
# print('*'*20)
# print(next(g))
# print('*'*25)
# print(next(g))

In [29]:
#搭建一个RNN model
import torch
import torch.nn as nn
class RNNModel(nn.Module):
    '''
    一个嵌入层，encoding
    一个循环神经网络层（RNN,LSTM,GRU）
    一个解码层，将隐层映射为输出
    一个dropout层，用于regularation。  
    '''
    def __init__(self,rnn_type,ntoken,ninp,nhid,nlayers,dropout=0.5):
        
        super(RNNModel,self).__init__()
        self.encode=nn.Embedding(ntoken,ninp)#生成词向量，ntoken表示输入样本个数，ninp表示向量长度
        self.dropout=nn.Dropout(dropout)
        #在搭建rnn层的时候，不是直接nn.RNN()，而是先看看rnn_type是否合适的！！
        if rnn_type in ['LSTM','GRU']:
            self.rnn=getattr(nn,rnn_type)(ninp,nhid,nlayers,dropout=dropout)
        else:
            try:
                nonlinearlity={'RNN_TANH':'tanh','RELU':'relu'}[rnn_type]
            except KeyError:
                raise ValueError(''' an valid option for "--model" was supplied , options are 
                                 ['LSTM','GRU','RNN_TANH','RNN_RELU']. ''')
            self.rnn=nn.RNN(ninp,nhid,nlayers,nonlinearity=nonlinearity,dropout=dropout)
        self.decode=nn.Linear(nhid,ntoken)
        self.init_weights()
        #将后面会需要用到的几个层加入到属性上来
        self.rnn_type=rnn_type
        self.nhid=nhid
        self.nlayers=nlayers
    def init_weights(self):
        #需要初始化的层是encoder和decoder。
        intrange=0.1
        self.encode.weight.data.uniform_(-intrange,intrange)
        self.decode.bias.data.zero_()
        self.decode.weight.data.uniform_(-intrange,intrange)
    def forward(self,input,hidden):
        emb=self.dropout(self.encode(input))
        output,hidden=self.rnn(emb,hidden)#注意不是直接像上面那样在这里直接就dropout，而只是对输出dropout。
        output=self.dropout(output)
        
        decoded=self.decode(output.view(output.size(0)*output.size(1),output.size(2)))
        return decoded.view(output.size(0),output.size(1),decoded.size(1)),hidden
    #初始化权重层，若是LSTM则比较特殊一点
    def init_hidden(self,bsz,requires_grad=True):
        weight=next(self.parameters())
        if self.rnn_type =='LSTM':
            return (weight.new_zeros((self.nlayers,bsz,self.nhid),requires_grad=requires_grad),
                   weight.new_zeros((self.nlayers,bsz,self.nhid),requires_grad=requires_grad))
        else:
            return weight.new_zeros((self.nlayers,bsz,self.nhid),requires_grad=requires_grad)
    
        

In [30]:
model=RNNModel('LSTM',VOCAB_SIZE,EMBEDDING_SIZE,EMBEDDING_SIZE,2,dropout=0.5)
if USE_CUDA:
    model=model.cuda()

In [31]:
'''
下面定义的这个函数可了不得了，，可以将一个tensor从一张计算图中分离出来，比如将隐层单独拿出来后，仅仅用于预测！！！
'''
def repackage_hidden(h):
    if isinstance(h,torch.torch.Tensor):
        return h.detach()
    else:
        return tuple(repackage_hidden(v) for v in h)

In [36]:
#模型评估代码，这个不是很熟悉，经常写模型训练代码，评估代码没注意过。
#这个地方比较特殊，在mode计算的时候，有一个hidden需要传入，但是之前的hidden
#是计算了梯度的，所以这里需要重新初始化一个hidden，不需要传入梯度，新生成的需要从原始图中摘除。
def evaluate(model,data):
    
    model.eval()
    total_count=0.
    total_loss=0.
    it=iter(data)
    
    with torch.no_grad():
        hidden=model.init_hidden(BATCH_SIZE,requires_grad=False)
        for i,batch in enumerate(it):
            data,target=batch.text,batch.target
            if USE_CUDA:
                data,target=data.cuda(),target.cuda()
            hidden=repackage_hidden(hidden)
            with torch.no_grad():
                output,hidden=model(data,hidden)
            loss=loss_fn(output.view(-1,VOCAB_SIZE),target.view(-1))
            total_count+=np.multiply(*data.size())
            total_loss+=loss.item()*np.multiply(*data.size())
    loss=total_loss/total_count
    model.train()
    return loss
            
        

In [37]:
data=torch.Tensor(1,2,3).cuda()
print(data.size())
print(np.multiply(*data.size()))

torch.Size([1, 2, 3])


TypeError: return arrays must be of ArrayType

In [38]:
loss_fn=torch.nn.CrossEntropyLoss()
lr=0.001
optimizer=torch.optim.Adam(model.parameters(),lr=lr)
scheduler=torch.optim.lr_scheduler.ExponentialLR(optimizer,0.5)#损失不下降的时候，就设置这个东西，让学习率下降。

In [43]:
#模型训练代码
GRAD_CLIP=1
NUM_EPOCHS=2
val_losses=[]
for epoch in range(NUM_EPOCHS):
    model.train()
    it=iter(train_iter)
    hidden=model.init_hidden(BATCH_SIZE)
    for i,batch in enumerate(it):
        data,target=batch.text,batch.target
        if USE_CUDA:
            data,target=data.cuda(),target.cuda()          
        hidden=repackage_hidden(hidden)
        
        model.zero_grad()
        
        output,hidden=model(data,hidden)
        
        loss=loss_fn(output.view(-1,VOCAB_SIZE),target.view(-1))
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(),GRAD_CLIP)
        optimizer.step()
        if i%1000==0:
            print('epoch',epoch,'iter',i,'loss',loss.item())
            
        if i%10000==0:
            val_loss=evaluate(model,val_iter)
            if len(val_losses)==0 or val_loss<min(val_losses):
                print('best model,val loss: ',val_loss)
                torch.save(model.state_dict(),'/home/control/Desktop/text8/lm-best.pth')
            else:
                sheduler.step()
                optimizer=torch.optim.Adam(model.parameters(),lr=lr)
            val_losses.append(val_loss)
    

epoch 0 iter 0 loss 5.883502006530762
best model,val loss:  5.257648636355273
epoch 0 iter 1000 loss 5.592380523681641
epoch 0 iter 2000 loss 5.630694389343262
epoch 0 iter 3000 loss 5.557417392730713
epoch 0 iter 4000 loss 4.943767547607422
epoch 0 iter 5000 loss 5.499133110046387
epoch 0 iter 6000 loss 5.375182151794434
epoch 0 iter 7000 loss 5.286401271820068
epoch 0 iter 8000 loss 5.407065391540527
epoch 0 iter 9000 loss 5.166967391967773
epoch 0 iter 10000 loss 5.30166482925415
best model,val loss:  5.017548044417916
epoch 0 iter 11000 loss 5.508561611175537
epoch 0 iter 12000 loss 5.507205963134766
epoch 0 iter 13000 loss 5.266716003417969
epoch 0 iter 14000 loss 5.1745285987854
epoch 1 iter 0 loss 5.521990776062012
best model,val loss:  4.958719471625291
epoch 1 iter 1000 loss 5.35995626449585
epoch 1 iter 2000 loss 5.473919868469238
epoch 1 iter 3000 loss 5.343467712402344
epoch 1 iter 4000 loss 4.811668872833252
epoch 1 iter 5000 loss 5.355369567871094
epoch 1 iter 6000 loss 5

In [45]:
best_model=RNNModel('LSTM',VOCAB_SIZE,EMBEDDING_SIZE,EMBEDDING_SIZE,2,dropout=0.5)
if USE_CUDA:
    best_model=best_model.cuda()
best_model.load_state_dict(torch.load('/home/control/Desktop/text8/lm-best.pth'))

<All keys matched successfully>

In [46]:
val_loss = evaluate(best_model,val_iter)
print('preplexity: ',np.exp(val_loss))

preplexity:  130.20918127859724


In [47]:
test_loss=evaluate(best_model,test_iter)
print('preplexity: ',np.exp(test_loss))

preplexity:  165.65304238321227
